In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

base_model = XGBClassifier(random_state=42, eval_metric='logloss')
base_model.fit(X_train, y_train)
base_pred = base_model.predict(X_test)

print(f"--- 🏁 튜닝 전 베이스라인 점수 ---")
print(f"정확도: {accuracy_score(y_test, base_pred):.4f}")

--- 🏁 튜닝 전 베이스라인 점수 ---
정확도: 0.9561


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)
score_rf = accuracy_score(y_test, pred_rf)
print(round(score_rf, 3))

report_rf = classification_report(y_test, pred_rf)
print(report_rf)

0.965
              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(estimator=XGBClassifier(eval_metric='logloss', random_state=42), param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print(f"\n--- 🏆 Grid Search 결과 ---")
print(f"최적의 파라미터: {grid_search.best_params_}")
print(f"최고의 점수(CV): {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_
final_pred = best_model.predict(X_test)
print(f"테스트셋 최종 정확도: {accuracy_score(y_test, final_pred):.4f}")


Fitting 3 folds for each of 27 candidates, totalling 81 fits

--- 🏆 Grid Search 결과 ---
최적의 파라미터: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
최고의 점수(CV): 0.9670
테스트셋 최종 정확도: 0.9561


In [ ]:
# 1. 라이브러리 설치 (없으면 실행)
# !pip install optuna 

import optuna
from sklearn.metrics import accuracy_score

# 2. 최적화할 목표 함수 정의 (Objective Function)
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 설정
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0), # 데이터 샘플링 비율
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0) # 컬럼 샘플링 비율
    }
    
    # 모델 생성 및 학습
    model = XGBClassifier(**params, random_state=42, eval_metric='logloss')
    model.fit(X_train, y_train)
    
    # 검증 (여기서는 편의상 Test셋 점수를 리턴하지만, 원래는 Validation셋을 써야 함)
    pred = model.predict(X_test)
    score = accuracy_score(y_test, pred)
    
    return score

# 3. 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize') # 점수를 '최대화' 해라
study.optimize(objective, n_trials=50) # 50번 시도해라

print(f"\n--- 🧠 Optuna 결과 ---")
print(f"가장 좋은 점수: {study.best_value:.4f}")
print(f"그때의 파라미터: {study.best_params}")

C:\Users\sangh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-17 17:34:54,867] A new study created in memory with name: no-name-a14eb8fd-e035-43f2-bdee-09679257aa9a
[I 2025-12-17 17:34:55,134] Trial 0 finished with value: 0.9736842105263158 and parameters: {'n_estimators': 256, 'max_depth': 7, 'learning_rate': 0.1668295070186331, 'subsample': 0.6421582021836753, 'colsample_bytree': 0.5643943719201644}. Best is trial 0 with value: 0.9736842105263158.
[I 2025-12-17 17:34:55,425] Trial 1 finished with value: 0.9736842105263158 and parameters: {'n_estimators': 252, 'max_depth': 7, 'learning_rate': 0.16573604264062927, 'subsample': 0.7645633641677422, 'colsample_bytree': 0.8881964935889421}. Best is tri


--- 🧠 Optuna 결과 ---
가장 좋은 점수: 0.9825
그때의 파라미터: {'n_estimators': 79, 'max_depth': 5, 'learning_rate': 0.1708439135326076, 'subsample': 0.576511712921509, 'colsample_bytree': 0.5095305664507473}


In [ ]:
from optuna.visualization import plot_optimization_history, plot_param_importances

# 1. 점수가 어떻게 올라가는지 보여줌 (우상향하면 성공!)
plot_optimization_history(study).show()

# 2. 어떤 파라미터가 점수에 가장 큰 영향을 줬는지 보여줌 (중요도)
plot_param_importances(study).show()

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import optuna

data = fetch_california_housing()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse'
    }

    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, pred, squared=False)

    return rmse

study_reg = optuna.create_study(direction='minimize')
study_reg.optimize(objective, n_trials=30)

print(f"\n--- 🏠 집값 예측 튜닝 결과 ---")
print(f"가장 낮은 오차(RMSE): {study_reg.best_value:.4f}")
print(f"최적의 파라미터: {study_reg.best_params}")

[I 2025-12-18 16:04:34,098] A new study created in memory with name: no-name-ad067215-9a5b-48a0-a774-3cf393841e3a
C:\Users\sangh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-12-18 16:04:39,319] Trial 0 finished with value: 0.4557594880897289 and parameters: {'n_estimators': 438, 'max_depth': 10, 'learning_rate': 0.10556759708021082, 'subsample': 0.896402464387477, 'colsample_bytree': 0.7164242695863545}. Best is trial 0 with value: 0.4557594880897289.
C:\Users\sangh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will


--- 🏠 집값 예측 튜닝 결과 ---
가장 낮은 오차(RMSE): 0.4375
최적의 파라미터: {'n_estimators': 324, 'max_depth': 8, 'learning_rate': 0.07783222604057137, 'subsample': 0.8049759264859953, 'colsample_bytree': 0.7633499328759792}


In [ ]:
from optuna.visualization import plot_optimization_history, plot_param_importances

# 1. 오차가 줄어드는 과정 (우하향해야 성공!)
plot_optimization_history(study_reg).show()

# 2. 집값 맞추는 데 가장 중요한 변수는? (중요도)
plot_param_importances(study_reg).show()